In [10]:
import numpy as np
import tkinter as tk
from PIL import Image, ImageDraw, ImageFont

In [11]:
class Element():
    def __init__(self, dofs, density):
        self.dofs = dofs
        self.d = density
        
    def calc_disp(self, global_U):
        self.u = np.mean(np.abs(global_U[self.dofs]))

In [30]:
class PostProcessor():
    def __init__(self, calc_name, low_color, high_color, bg_color, outline, draw_displ):
        self.window_name = calc_name
        self.filename = calc_name + '.txt'
        self.low_col = low_color
        self.high_col = high_color
        self.bg_col = bg_color
        self.out = outline
        self.draw_u = draw_displ
        
    def parse_file(self):
        in_file = open(self.filename, "r")
        
        str_data = in_file.read()
        sections = str_data.split('-section-\n')
        
        mesh_size = sections[0].replace('\n', '').split(' ')
        self.nx = int(mesh_size[0])
        self.ny = int(mesh_size[1])
        
        self.displacements = np.array(list(map(lambda x: float(x), sections[1].split('\n')[:-1])))
        self.fixed_dofs = np.array(list(map(lambda x: int(x), sections[2].split('\n')[:-1])))
        self.densities = np.array(list(map(lambda x: float(x), sections[3].split('\n')[:-1])))
        
    def map_value(self, x, in_min, in_max, out_min, out_max):
        return ((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)  
    
    def map_color(self, low_val, high_val, val):
        low_r, low_g, low_b = self.low_col
        high_r, high_g, high_b = self.high_col
        
        r = int(self.map_value(val, low_val, high_val, low_r, high_r))
        g = int(self.map_value(val, low_val, high_val, low_g, high_g))
        b = int(self.map_value(val, low_val, high_val, low_b, high_b))
        
        return (r,g,b)
    
    def form_elements(self):
        self.ex = self.nx - 1
        self.ey = self.ny - 1
        self.ne = self.ey * self.ex
        self.elements = []
        
        for i in range(self.ne):
            lb_i = self.ny * (i // self.ey) + (i % self.ey) + 1 # index of left bottom node
            
            elem_dofs = np.array([
                2*lb_i, 2*lb_i + 1,
                2*(lb_i + self.ny), 2*(lb_i + self.ny) + 1,
                2*(lb_i + self.ny - 1), 2*(lb_i + self.ny - 1) + 1,
                2*(lb_i - 1), 2*(lb_i - 1) + 1
            ])
            
            new_element = Element(elem_dofs, self.densities[i])
            new_element.calc_disp(self.displacements)
            self.elements.append(new_element)
            
    def extremal_displ(self):
        self.min_u = 1e9
        self.max_u = 0
        
        for e in self.elements:
            if (e.u > self.max_u):
                self.max_u = e.u
            if (e.u < self.min_u):
                self.min_u = e.u
            
            
    def prepare_canvas(self):
        #self.window = tk.Tk()
        #self.window.title(self.window_name)
        
        #self.canv = tk.Canvas(self.window, width = 900, height = 900, bg = self.bg_col)
        #self.canv.pack()
        self.canv = Image.new('RGB', (900, 900), self.bg_col)
        self.draw = ImageDraw.Draw(self.canv)
        self.font = ImageFont.truetype('font_1.ttf', 18)
        
    def from_rgb(self, rgb):
        r, g, b = rgb
        return (f'#{r:02x}{g:02x}{b:02x}')
        
    def draw_grid(self, n, grid_color):
        step = 900 / n

        for i in range(n):
            #self.canv.create_line(0, i*step, 900, i*step, fill = self.from_rgb(grid_color))
            #self.canv.create_line(i*step, 0, i*step, 900, fill = self.from_rgb(grid_color))
            self.draw.line((0, i*step, 900, i*step), fill = grid_color)
            self.draw.line((i*step, 0, i*step, 900), fill = grid_color)
    
    def draw_elements(self, margin, dense_threshold):
        for i in range(self.ne):
            density = self.elements[i].d
            disp = self.elements[i].u
            elem_u = self.displacements[self.elements[i].dofs]
            
            e_col = i // self.ey
            e_row = i % self.ey
            
            lb_col = e_col
            lb_row = e_row + 1
            
            elem_coords = [lb_col, lb_row, 
                           lb_col + 1, lb_row, 
                           lb_col + 1, lb_row - 1,
                           lb_col, lb_row - 1]
            
            elem_coords_mapped = []
            if (self.nx >= self.ny):
                x_area = 900 - 2*margin
                y_area = int(x_area * self.ny / self.nx)
                y_margin = int((900 - y_area)/2)
                
                for i in range(4):
                    elem_coords_mapped.append(self.map_value(elem_coords[2*i], 0, self.nx, margin, 900 - margin))
                    elem_coords_mapped.append(self.map_value(elem_coords[2*i + 1], 0, self.ny, y_margin, 900 - y_margin))
                    
            else:
                y_area = 900 - 2*margin
                x_area = int(y_area * self.nx / self.ny)
                x_margin = int((900 - x_area)/2)
                
                for i in range(4):
                    elem_coords_mapped.append(self.map_value(elem_coords[2*i], 0, self.nx, x_margin, 900 - x_margin))
                    elem_coords_mapped.append(self.map_value(elem_coords[2*i + 1], 0, self.ny, margin, 900 - margin))
            if (self.draw_u == True):
                for i in range(8):
                    elem_coords_mapped[i] += self.map_value(disp, self.min_u, self.max_u, 0, 30)
            if (density > dense_threshold):
                color = self.from_rgb(self.map_color(self.min_u, self.max_u, disp))
                self.draw.polygon(elem_coords_mapped, fill = color, outline = color if self.out == False else 'black', width = 1)
    
    def draw_legend(self):
        n = 30
        delta_pix = 5

        value_grid = np.linspace(self.min_u, self.max_u, n)
        width = 40
        bottom_pix = 300

        for i in range(n):
            color = self.from_rgb(self.map_color(self.min_u, self.max_u, value_grid[i]))
            self.draw.rectangle((800, 
                                bottom_pix - i * delta_pix, 
                                800 + width, 
                                bottom_pix - (i-1)*delta_pix), 
                                fill = color,
                                outline = color)
        
        font = ImageFont.truetype('font_1.ttf', 12)
        self.draw.text((845, bottom_pix - 10), text = str(round(self.min_u, 2)), fill = (0,0,0), font = font)
        self.draw.text((845, bottom_pix - n*delta_pix), text = str(round(self.max_u,2)), fill = (0,0,0), font = font)
        
    def add_title(self, title):
        font = ImageFont.truetype('font_1.ttf', 24)
        self.draw.text((370, 80), text = title, font = font, fill = (0,0,0))
        
    def add_subtitle(self, subtitle):
        font = ImageFont.truetype('font_1.ttf', 18)
        self.draw.text((450, 120), text = subtitle, font = font, fill = (0,0,0))
        
    def visualize(self):
        self.parse_file()
        self.form_elements()
        self.extremal_displ()
        self.prepare_canvas()
        
        grid_color = tuple([200, 210, 240])
        
        self.draw_grid(15, grid_color)
        self.draw_elements(60, 0.1)
        self.draw_legend()
        self.add_title('Design_1')
        self.add_subtitle(self.window_name)
        self.canv.save(self.window_name + '.png')
        self.canv.show()
        

In [34]:
low_color = (240, 220, 100)
high_color = (100, 240, 240)

post_processor = PostProcessor('Iter_35', low_color, high_color, (255,255,255), False, False)
post_processor.visualize()